In [ ]:
import pandas as pd
import numpy as np
from sklearn import svm
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
file_name='/home/corinnet/workspace/aiephys/multipatch_analysis/analyses/ML_connected_vclamp_2018_12_12.csv'

df=pd.read_csv(file_name)
print(df.keys())

In [ ]:
subset=df[['NRMSE', 'amp', 'decay_tau', 'latency', 'n_sweeps', 'good_fit', 'data_clarity']]
g=sns.pairplot(subset, 
             vars=['NRMSE', 'amp', 'decay_tau', 'latency', 'n_sweeps'], 
             hue='good_fit')

In [ ]:
subset=df[['NRMSE', 'amp', 'decay_tau', 'latency', 'n_sweeps', 'good_fit', 'data_clarity']]
g=sns.pairplot(subset, 
             vars=['NRMSE', 'amp', 'decay_tau', 'latency', 'n_sweeps'], 
             hue='data_clarity')

In [ ]:
#Based on images above nrmse and amplitude may help classify good versus bad fits
subset=df[(df.good_fit=='excellent') | (df.good_fit=='bad')]

def numeric_catagory(row):
    if row['good_fit'] == 'excellent':
        return int(1)
    elif row['good_fit'] == 'bad':
        return int(0)
#add a new column with a number as the lable for plotting below
subset['numerical']=""
subset['numerical']=subset.apply(lambda row: numeric_catagory(row), axis=1)
X=subset[['NRMSE', 'amp']].values
y=subset['numerical'].values

In [ ]:
subset['numerical']

In [ ]:
# from sklearn.datasets.samples_generator import make_blobs
# X, y = make_blobs(n_samples=50, centers=2,
#                   random_state=0, cluster_std=0.60)
# plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn');

clf=svm.SVC()
clf.fit(X, y)

In [ ]:
def plot_svc_decision_function(model, ax=None, plot_support=True):
    """Plot the decision function for a 2D SVC"""
    if ax is None:
        ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # create grid to evaluate model
    x = np.linspace(xlim[0], xlim[1], 30)
    y = np.linspace(ylim[0], ylim[1], 30)
    Y, X = np.meshgrid(y, x)
    xy = np.vstack([X.ravel(), Y.ravel()]).T
    P = model.decision_function(xy).reshape(X.shape)
    
    # plot decision boundary and margins
    ax.contour(X, Y, P, colors='k',
               levels=[-1, 0, 1], alpha=0.5,
               linestyles=['--', '-', '--'])
    
    # plot support vectors
    if plot_support:
        ax.scatter(model.support_vectors_[:, 0],
                   model.support_vectors_[:, 1],
                   s=300, linewidth=1, facecolors='none');
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

def make_meshgrid(x, y, h=.02):
    """Create a mesh of points to plot in

    Parameters
    ----------
    x: data to base x-axis meshgrid on
    y: data to base y-axis meshgrid on
    h: stepsize for meshgrid, optional

    Returns
    -------
    xx, yy : ndarray
    """
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy


def plot_contours(ax, clf, xx, yy, **params):
    """Plot the decision boundaries for a classifier.

    Parameters
    ----------
    ax: matplotlib axes object
    clf: a classifier
    xx: meshgrid ndarray
    yy: meshgrid ndarray
    params: dictionary of params to pass to contourf, optional
    """
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

In [ ]:
# we create an instance of SVM and fit out data. We do not scale our
# data since we want to plot the support vectors
C = 1.0  # SVM regularization parameter
models = (svm.SVC(kernel='linear', C=C),
          svm.LinearSVC(C=C),
          svm.SVC(kernel='rbf', gamma=0.7, C=C),
          svm.SVC(kernel='poly', degree=3, C=C))
models = (clf.fit(X, y) for clf in models)

# title for the plots
titles = ('SVC with linear kernel',
          'LinearSVC (linear kernel)',
          'SVC with RBF kernel',
          'SVC with polynomial (degree 3) kernel')

# Set-up 2x2 grid for plotting.
fig, sub = plt.subplots(2, 2)
plt.subplots_adjust(wspace=0.4, hspace=0.4)

X0, X1 = X[:, 0], X[:, 1]
xx, yy = make_meshgrid(X0, X1)

for clf, title, ax in zip(models, titles, sub.flatten()):
    print(clf, title, ax)
#    plot_contours(ax, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
    ax.scatter(X0, X1, c=y, cmap=plt.cm.coolwarm, s=20, edgecolors='k')
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xlabel('Sepal length')
    ax.set_ylabel('Sepal width')
    ax.set_xticks(())
    ax.set_yticks(())
    ax.set_title(title)

In [ ]:
X